In [1]:
# For Reproducable Code
from numpy.random import seed
seed(42)
import tensorflow
tensorflow.random.set_seed(42)
from cupy.random import seed
seed(42)

# other standard imports. "pip install -r requirements.txt" to install dependencies and "pip freeze > requirements.txt" to update them
from collections import Counter
import pickle

import PIL
import scipy
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import plotly.express as px

from sklearn.metrics import accuracy_score
from dbn import SupervisedDBNClassification

In [2]:
# Useful Functions
def pklSave(contentToBeSaved, fullPath):
    with open(fullPath, 'wb') as f:
        pickle.dump(contentToBeSaved, f)

def pklLoad(fullPath, convertToNumpyArray=False):
    with open(fullPath, 'rb') as f:
        content = pickle.load(f)
    if convertToNumpyArray:
        content = np.array(content)
    return content

def train_val_test_split(x, y, tr, va, te, **kwargs):
    x_train, x_test_val, y_train, y_test_val = train_test_split(x, y, test_size=1-tr)
    x_val, x_test, y_val, y_test = train_test_split(x_test_val, y_test_val, test_size=te/(te+va), **kwargs)
    return (x_train, x_val, x_test, y_train, y_val, y_test)

In [ ]:
def custom_learning_curve(estimator, x, y, train_sizes = None, test_sizes = None): # pass x, y as lists or nparrays
    if train_sizes is None:
        train_sizes = [0.2,0.4,0.6,0.8,1]
    if test_sizes is None:
        test_sizes = [0.2,0.2,0.2,0.2,0.2]
    
    train_accs = []
    test_accs = []

    if type(estimator) == SupervisedDBNClassification:
        for i in range(len(train_sizes)):
            if train_sizes[i] == 1:
                x_new, y_new = x, y
            else:
                x_new, _, y_new, _ = train_test_split(x, y, test_size=1-train_sizes[i], random_state=42)

            x_train, x_test, y_train, y_test = train_test_split(x_new, y_new, test_size=test_sizes[i], random_state=42)
            x_train = cp.array(x_train)
            x_test = cp.array(x_test)
            y_train = cp.array(y_train) 
            y_test = cp.array(y_test)

            estimator.fit(x_train, y_train)

            y_pred_train = np.array(estimator.predict(x_train))
            y_pred_test = np.array(estimator.predict(x_test))
            y_train = np.array(y_train.get()) #converts cp array to np array to be compatible with accuracy_score
            y_test = np.array(y_test.get())

            train_accs.append(accuracy_score(y_train, y_pred_train))
            test_accs.append(accuracy_score(y_test, y_pred_test))
        return (train_sizes, train_accs, test_accs)

In [ ]:
dfsTrAccs = [] # Tr == Training
dfsTsAccs = [] # Ts == Testing

# Fitting the model on Rescaled Images

In [ ]:
x = pklLoad('../x128.pkl', convertToNumpyArray=True)
y = pklLoad('../y.pkl', convertToNumpyArray=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train = cp.array(x_train)
x_test = cp.array(x_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [ ]:
len(x_train), len(x_test), len(y_train), len(y_test)

(1436, 360, 1436, 360)

In [ ]:
x_train.shape

(1436, 16384)

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.1, #0.05 with 50 epochs == 0.1 with 20 epochs
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5, # loss was found to be stagnating after this value
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1) # low drop-out value, as model is underfitting

In [ ]:
# classifier.fit(x_train, y_train)

Fitting Results:
<br><br>
<img src="notebook_media/dbn128training.png" height=800 />

In [ ]:
# classifier.save('dbnModel128.pkl')

In [ ]:
classifier = SupervisedDBNClassification.load('dbnModel128.pkl')

In [ ]:
def printDbnResults(classifier, trAccs=None, tsAccs=None):
    """Note: assumes you globally defined x_train/test & y_train/test before calling the function"""
    y_pred = classifier.predict(x_train) # predict returns a list
    y_pred = np.array(y_pred) # converting to nparray to be compatible with accuracy_score
    y_train_np = np.array(y_train.get())
    y_test_np = np.array(y_test.get())
    dbn_train_score = accuracy_score(y_train_np, y_pred)
    print(f'Training Accuracy: {dbn_train_score}')
    y_pred = classifier.predict(x_test)
    y_pred = np.array(y_pred)
    dbn_test_score = accuracy_score(y_test_np, y_pred)
    print(f'Testing Accuracy: {dbn_test_score}')

    trAccs.append(dbn_train_score)
    tsAccs.append(dbn_test_score)

In [ ]:
printDbnResults(classifier, dfsTrAccs, dfsTsAccs)

Training Accuracy: 0.387883008356546
Testing Accuracy: 0.33055555555555555


# Fitting the model on Features Extracted From the Images

In [ ]:
xf = pklLoad('../x128f.pkl', convertToNumpyArray=True)
y = pklLoad('../y.pkl', convertToNumpyArray=True)
x_train, x_test, y_train, y_test = train_test_split(xf, y, test_size=0.2, random_state=42)
x_train = cp.array(x_train)
x_test = cp.array(x_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1)

In [ ]:
# classifier.fit(x_train, y_train)

Fitting Results: <br>
<img src='notebook_media/dbn128ftraining.png' height=850 />

In [ ]:
# classifier.save('dbnModel128f.pkl')

In [ ]:
classifier = SupervisedDBNClassification.load('dbnModel128f.pkl')

In [ ]:
printDbnResults(classifier, dfsTrAccs, dfsTsAccs)

Training Accuracy: 0.19707520891364902
Testing Accuracy: 0.21388888888888888


## on 64x64

In [ ]:
xf = pklLoad('../x64f.pkl', convertToNumpyArray=True)
y = pklLoad('../y.pkl', convertToNumpyArray=True)
x_train, x_test, y_train, y_test = train_test_split(xf, y, test_size=0.2, random_state=42)
x_train = cp.array(x_train)
x_test = cp.array(x_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)

Fitting Results: <br>
<img src='notebook_media/dbn64ftraining.png' height=850 />

In [ ]:
# classifier.save('dbnModel64f.pkl')

In [ ]:
classifier = SupervisedDBNClassification.load('dbnModel64f.pkl')

In [ ]:
printDbnResults(classifier, dfsTrAccs, dfsTsAccs)

Training Accuracy: 0.19498607242339833
Testing Accuracy: 0.2222222222222222


# Fitting the model on Augmented Images

In [8]:
x128aug = pklLoad('../x128aug.pkl', convertToNumpyArray=True)
y = pklLoad('../yaug.pkl', convertToNumpyArray=True)
x_train, x_test, y_train, y_test = train_test_split(x128aug, y, test_size=0.2, random_state=42)
x_train = cp.array(x_train)
x_test = cp.array(x_test)
y_train = cp.array(y_train)
y_test = cp.array(y_test)

In [9]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [10]:
classifier.fit(x_train, y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 1326.921285
>> Epoch 2 finished 	RBM Reconstruction error 1285.241055
>> Epoch 3 finished 	RBM Reconstruction error 1261.074353
>> Epoch 4 finished 	RBM Reconstruction error 1238.980367
>> Epoch 5 finished 	RBM Reconstruction error 1230.303326
>> Epoch 6 finished 	RBM Reconstruction error 1226.663819
>> Epoch 7 finished 	RBM Reconstruction error 1219.086776
>> Epoch 8 finished 	RBM Reconstruction error 1212.321727
>> Epoch 9 finished 	RBM Reconstruction error 1199.365882
>> Epoch 10 finished 	RBM Reconstruction error 1187.645742
>> Epoch 11 finished 	RBM Reconstruction error 1164.140018
>> Epoch 12 finished 	RBM Reconstruction error 1142.155379
>> Epoch 13 finished 	RBM Reconstruction error 1114.130430
>> Epoch 14 finished 	RBM Reconstruction error 1064.555405
>> Epoch 15 finished 	RBM Reconstruction error 1022.636188
>> Epoch 16 finished 	RBM Reconstruction error 980.746360
>> Epoch 17 finished 	RBM Reconstructio

SupervisedDBNClassification()

Fitting Result: <br>
<img src='notebook_media/dbn128augtraining.png' height=850 />

In [ ]:
y_pred = classifier.predict(x_train) # predict returns a list
y_pred = np.array(y_pred) # converting to nparray to be compatible with accuracy_score
y_train_np = np.array(y_train.get())
y_test_np = np.array(y_test.get())
dbn_train_score = accuracy_score(y_train_np, y_pred)
print(f'Training Accuracy: {dbn_train_score}')
y_pred = classifier.predict(x_test)
y_pred = np.array(y_pred)
dbn_test_score = accuracy_score(y_test_np, y_pred)
print(f'Testing Accuracy: {dbn_test_score}')

In [ ]:
# classifier.save('dbnModel128aug.pkl')

In [ ]:
classifier = SupervisedDBNClassification.load('dbnModel128aug.pkl')

In [ ]:
printDbnResults(classifier, dfsTrAccs, dfsTsAccs)

Training Accuracy: 0.4668522707499565
Testing Accuracy: 0.46903270702853167


In [6]:
# dfsNames = ["128x128", "1.5K PCA Extracted Features from 128x128", "1.25K PCA Extracted Features from 64x64", "128x128 Augmented (4-Folds)"]
# pklSave((dfsNames, dfsTrAccs, dfsTsAccs), "dfsDbnAccs.pickle")
dfsNames, dfsTrAccs, dfsTsAccs = pklLoad("dfsDbnAccs.pickle")
dfsNames, dfsTrAccs, dfsTsAccs

(['128x128',
  '1.5K PCA Extracted Features from 128x128',
  '1.25K PCA Extracted Features from 64x64',
  '128x128 Augmented (4-Folds)'],
 [0.387883008356546,
  0.19707520891364902,
  0.19498607242339833,
  0.4668522707499565],
 [0.33055555555555555,
  0.21388888888888888,
  0.2222222222222222,
  0.46903270702853167])

In [7]:
def custom_legend_name(fig, new_names):
    for i, new_name in enumerate(new_names):
        fig.data[i].name = new_name
    return fig

In [8]:
fig = px.line(x=dfsNames, y=[dfsTrAccs, dfsTsAccs], markers=True)
fig.update_layout(title="Comparing DBN Performance on Different Pre-Processed Datasets", legend_title="Accuracy Type")
fig.update_layout(xaxis={"title": "Alteration Done on Images"})
fig = custom_legend_name(fig, ["Training Accuracy", "Testing Accuracy"])
fig.show()

# Hypertuning on 128-Aug. Dataset

As we can see, the augmented dataset on the rescaled 128x128 images yielded the best results, given that all DBN models had the same hyper-parameters. <br>
Therefore, let's further train the last DBN model on the augmented dataset with different hyper-parameters.
<br>
Recall that this was our initial 128AugDBN Hyper-Parameters: 
<br><br>
<img src="notebook_media/dbn128augparams.png" width=300 />
<br><br>
with this fitting result:
<br><br>
<img src="notebook_media/dbn128augtraining.png" height=850 />
<br><br>
Which gave out these accuracies: <br>
0.4668522707499565 Training Accuracy <br>
0.46903270702853167 Testing Accuracy

In [ ]:
df128augTrAccs = [dfsTrAccs[-1]]
df128augTsAccs = [dfsTsAccs[-1]]

## Changing RBM's Learning Rate
It makes sense that this is the first hyper-parameter to tune, since previously, we saw high RBM reconstruction errors that slowly converged 

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.15,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)
# classifier.save('dbnModel128augRbmLR.pkl')
classifier = SupervisedDBNClassification.load('dbnModel128augRbmLR.pkl')
printDbnResults(classifier, df128augTrAccs, df128augTsAccs)

Training Accuracy: 0.464068209500609
Testing Accuracy: 0.4349338900487126


Fitting Result: <br>
<img src='notebook_media/dbn128augRbmLRtraining.png' height=850 />

## Changing NN's Learning Rate
The training loss at the fine-tuning (back-propagation) step was also slowly converging, so let's increase its learning rate!

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.15,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)
# classifier.save('dbnModel128augNNLR.pkl')
classifier = SupervisedDBNClassification.load('dbnModel128augNNLR.pkl')
printDbnResults(classifier, df128augTrAccs, df128augTsAccs)

Training Accuracy: 0.47711849660692535
Testing Accuracy: 0.4516353514265832


Fitting Result: <br>
<img src='notebook_media/dbn128augNNLRtraining.png' height=850 />

## Changing #Hidden-Layers
This alteration could negligibly affect the model's performance, as we saw very low reconstruction errors in the second RBM, so adding a third one won't probably be that beneficial

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128, 128],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)
# classifier.save('dbnModel128aug3L.pkl')
classifier = SupervisedDBNClassification.load('dbnModel128aug3L.pkl')
printDbnResults(classifier, df128augTrAccs, df128augTsAccs)

Training Accuracy: 0.4764224812945885
Testing Accuracy: 0.4704244954766875


Fitting Result: <br>
<img src='notebook_media/dbn128aug3Ltraining.png' height=850 />

## Changing #Hidden-Neurons

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[256, 256],
                                         learning_rate_rbm=0.1,
                                         learning_rate=0.1,
                                         n_epochs_rbm=20,
                                         n_iter_backprop=5,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)
# classifier.save('dbnModel128aug256HN.pkl')
classifier = SupervisedDBNClassification.load('dbnModel128aug256HN.pkl')
printDbnResults(classifier, df128augTrAccs, df128augTsAccs)

Training Accuracy: 0.4082129806855751
Testing Accuracy: 0.4036186499652053


Fitting Result: <br>
<img src='notebook_media/dbn128aug256HNtraining.png' height=850 />

## Changing RBM & Back Propagation's #Epochs 
We'll also decrease the RBM's learning by half, since we increased RBM's #Epocs by the double

In [ ]:
classifier = SupervisedDBNClassification(hidden_layers_structure=[128, 128],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=60,
                                         n_iter_backprop=10,
                                         batch_size=32,
                                         activation_function='sigmoid',
                                         dropout_p=0.1,
                                         verbose=True)

In [ ]:
# classifier.fit(x_train, y_train)
# classifier.save('dbnModel128augEpochsAndRbmLR.pkl')
classifier = SupervisedDBNClassification.load('dbnModel128augEpochsAndRbmLR.pkl')
printDbnResults(classifier, df128augTrAccs, df128augTsAccs)

Training Accuracy: 0.4939968679310945
Testing Accuracy: 0.46555323590814196


Due to the large output, no screenshot of the errors is provided. <br>
Either way, the final RBM & NN errors were similar to the original DBN used on 128-Aug dataset <br>**except** the first layer's RBM reconstruction error which converged to approximately `500` instead of `900`

# Comparing The Hyper-Tuned Models

In [9]:
htDbnsNames = [
    "HL:[128HN, 128HN], RBM/NN LR: 0.1, RBM #Epochs: 20, BP #Epochs: 5", 
    "RBM LR: 0.3", 
    "NN LR: 0.3", 
    "HL: [128,128,128]", 
    "HL: [256,256]", 
    "RBM #Epochs: 100, NN #Epochs: 20, RBM LR: 0.05"
]
# pklSave((htDbnsNames, df128augTrAccs, df128augTsAccs), "df128augAccs.pickle")
dfsNames, df128augTrAccs, df128augTsAccs = pklLoad("df128augAccs.pickle")
dfsNames, df128augTrAccs, df128augTsAccs

(['HL:[128HN, 128HN], RBM/NN LR: 0.1, RBM #Epochs: 20, BP #Epochs: 5',
  'RBM LR: 0.3',
  'NN LR: 0.3',
  'HL: [128,128,128]',
  'HL: [256,256]',
  'RBM #Epochs: 100, NN #Epochs: 20, RBM LR: 0.05'],
 [0.4668522707499565,
  0.464068209500609,
  0.47711849660692535,
  0.4764224812945885,
  0.4082129806855751,
  0.4939968679310945],
 [0.46903270702853167,
  0.4349338900487126,
  0.4516353514265832,
  0.4704244954766875,
  0.4036186499652053,
  0.46555323590814196])

In [10]:
fig = px.line(x=dfsNames, y=[df128augTrAccs, df128augTsAccs], markers=True)
fig.update_layout(title="Comparing Different DBNs on 128x128 4-Folds Augmented Dataset", legend_title="Accuracy Type")
fig.update_layout(xaxis={"title": "Alteration Done on DBN's Hyperparameters (1st one is the original)"})
fig = custom_legend_name(fig, ["Training Accuracy", "Testing Accuracy"])
fig.show()

As you can see, the performance increased when we <font color="red"> increased blah blah blah </font>
<br>
Therefore, we'll save this model along with its results as the final DBN model to be trained and compared with the other ANN models

In [ ]:
classifier = SupervisedDBNClassification.load('dbnModel128augEpochsAndRbmLR.pkl')
#pklSave(classifier, "finalDbn.pickle")
#pklSave((df128augTrAccs[-1], df128augTsAccs[-1]), "finalDbnTrTsAccs.pickle")

In [ ]:
# dbn = pklLoad("DBN/finalDbn.pickle")
# dbnTrAcc, dbnTsAcc = pklLoad("DBN/finalDbnTrTsAccs.pickle")